In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-02-21 12:17:23--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  80.6MB/s    in 0.3s    

2025-02-21 12:17:23 (80.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [36]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

#Create Dataset

In [37]:
count_of_rating = df_ratings.groupby('isbn')['rating'].count().rename(columns={'rating': 'count_of_ratings'})
avg_of_rating = df_ratings.groupby('isbn')['rating'].mean().rename(columns={'rating': 'avg_of_ratings'})
df_avg_ratings = pd.merge(count_of_rating, avg_of_rating, on='isbn')

In [38]:
df_avg_ratings

,rating_x,rating_y
isbn,,
0330299891,2,3.0
0375404120,2,1.5
0586045007,1,0.0
9022906116,2,3.5
9032803328,1,0.0
...,...,...
cn113107,1,0.0
ooo7156103,1,7.0
§423350229,1,0.0


In [39]:
df = pd.merge(df_books, df_avg_ratings, on='isbn')

In [40]:
df.sort

,isbn,title,author,rating_x,rating_y
0,0195153448,Classical Mythology,Mark P. O. Morford,1,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,14,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,3,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,11,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1,0.000000
...,...,...,...,...,...
270165,0440400988,There's a Bat in Bunk Five,Paula Danziger,1,7.000000
270166,0525447644,From One to One Hundred,Teri Sloat,1,4.000000
270167,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,1,0.000000
270168,0192126040,Republic (World's Classics),Plato,1,0.000000


In [30]:
# add your code here - consider creating a new cell for each section of code
df_books.sort_values(by=['author'])

,isbn,title,author
144056,8423920143,"GuÃ­a del lector del \Quijote\"": Ensayo psicol...","14 : Ensayo)"""
104317,0373166982,"Please Say \I Do\"" (Three Weddings &amp","A Hurricane) (Harlequin American Romance, No ..."
180187,0553570722,"\R\"" for Revenge: Written by Kate William",Created by Francine Pascal (Sweet Valley High...
18119,0312956762,Die Hard: With a Vengeance - A Novel,D. Chiel
251294,0723245827,\The World of Peter Rabbit &amp,"Friends\"" Complete Story Collection"""
...,...,...,...
191525,3518188070,"Suhrkamp BasisBibliothek (SBB), Nr.7, Jugend o...",Ã?Â?dÃ?Â¶n von Horvath
139824,2268045757,Maximes et pensÃ?Â©es : Ã?Â?pictÃ?Â¨te,Ã?Â?pictÃ?Â¨te
33915,2290048763,Mademoiselle Chambon,Ã?Â?ric Holder
118038,0751352497,A+ Quiz Masters:01 Earth,NaN


In [29]:
df_ratings.sort_values(by=['rating'])

,user,isbn,rating
0,276725,034545104X,0.0
687400,167471,0806515082,0.0
687402,167471,0806958855,0.0
687403,167471,080698323X,0.0
687407,167471,0811841405,0.0
...,...,...,...
272376,63625,0316168815,10.0
222656,52044,0486287297,10.0
272375,63625,0312147015,10.0
406256,98391,0312252617,10.0


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()